### Dataset

In [1]:
from datasets import load_dataset

DATASET_NAME = "DiscoResearch/germanrag"
dataset = load_dataset(DATASET_NAME, split="train")

Using the latest cached version of the dataset since DiscoResearch/germanrag couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/shawon/.cache/huggingface/datasets/DiscoResearch___germanrag/default/0.0.0/dcaafc4bf5b44c96a1ced56c0a7d42da5a3a2c45 (last modified on Mon Feb 26 23:09:34 2024).


### Find all unique contexts

In [2]:
from tqdm.autonotebook import tqdm

unique_contexts = set()
for _, d in tqdm(enumerate(dataset), total=len(dataset)):
    for ctx in d["contexts"]:
        unique_contexts.add(ctx)

  0%|          | 0/3362 [00:00<?, ?it/s]

### Easy Negative Finder

In [3]:
from src.docs import EasyNegativeFinder
from loguru import logger
import torch

model_name = "sentence-transformers/distiluse-base-multilingual-cased-v2"
device = "cuda" if torch.cuda.is_available() else "cpu"
# path to persist the vector store
dpath = "./persisted_data"

logger.info("Creating Finder Object")
finder = EasyNegativeFinder(model_name, device, dpath)

2024-02-28 22:32:40.810 | INFO     | __main__:<module>:10 - Creating Finder Object
2024-02-28 22:32:40.811 | INFO     | src.docs:__load_embedding_model:18 - Initialising Embeddings with model ::sentence-transformers/distiluse-base-multilingual-cased-v2
2024-02-28 22:34:53.659 | SUCCESS  | src.docs:__load_embedding_model:23 - Done


In [4]:
# init vector store
finder.init_vector_store(list(unique_contexts))

2024-02-28 22:34:53.708 | INFO     | src.docs:init_vector_store:32 - Populating Vector Store
2024-02-28 22:34:53.709 | INFO     | src.docs:init_vector_store:33 - Persisted @ :: ./persisted_data
2024-02-28 22:35:10.774 | SUCCESS  | src.docs:init_vector_store:37 - Done


### Prepare dataset in the desired format

In [6]:
from src.dataset import prepare_entire_dataset

prepared_dataset = prepare_entire_dataset(dataset, finder)
logger.info(f"Size of the prepared dataset : {len(prepared_dataset)}")

2024-02-28 22:35:41.087 | INFO     | src.dataset:prepare_entire_dataset:57 - Converting Dataset into JSONL format


  0%|          | 0/3362 [00:00<?, ?it/s]

2024-02-28 22:36:14.164 | SUCCESS  | src.dataset:prepare_entire_dataset:65 - Done
2024-02-28 22:36:14.165 | INFO     | __main__:<module>:4 - Size of the prepared dataset : 11197


In [7]:
prepared_dataset[:10]

[{'context': 'Gott\n\n=== Demografie ===\nEine Zusammenfassung von Umfrageergebnissen aus verschiedenen Staaten ergab im Jahr 2007, dass es weltweit zwischen 505 und 749 Millionen Atheisten und Agnostiker gibt. Laut der Encyclopædia Britannica gab es 2009 weltweit 640 Mio. Nichtreligiöse und Agnostiker (9,4 %), und weitere 139 Mio. Atheisten (2,0 %), hauptsächlich in der Volksrepublik China.\nBei einer Eurobarometer-Umfrage im Jahr 2005 wurde festgestellt, dass 52 % der damaligen EU-Bevölkerung glaubt, dass es einen Gott gibt. Eine vagere Frage nach dem Glauben an „eine andere spirituelle Kraft oder Lebenskraft“ wurde von weiteren 27 % positiv beantwortet. Bezüglich der Gottgläubigkeit bestanden große Unterschiede zwischen den einzelnen europäischen Staaten. Die Umfrage ergab, dass der Glaube an Gott in Staaten mit starkem kirchlichen Einfluss am stärksten verbreitet ist, dass mehr Frauen (58 %) als Männer (45 %) an einen Gott glauben und dass der Gottglaube mit höherem Alter, geringer

### Next Steps

Write the prepared dataset to disk so that it can be used later. Another approach
can be to find easy negatives on demand in a data loading pipeline so that instead of finding all easy negatives in one pass, we only find easy negatives for data instances
from the dataset we currently need. 